In [11]:
%pip install requests

  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
                                              0.0/99.9 kB ? eta -:--:--
     ---------------------------------------- 99.9/99.9 kB 2.8 MB/s eta 0:00:00
                                              0.0/104.6 kB ? eta -:--:--
     -------------------------------------- 104.6/104.6 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
import random
import openai
import pandas as pd
openai.api_key = #Enter your OpenAPI key

C:\Users\DELL\AppData\Local\Temp\ipykernel_9520\3759517043.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
story_types_list = [
    "Adventure",
    "Mystery",
    "Romance",
    "Science Fiction",
    "Fantasy",
    "Historical Fiction",
    "Thriller",
    "Horror",
    "Comedy",
    "Drama",
    "Action",
    "Crime",
    "Western",
    "Children's",
    "Young Adult",
    "Dystopian",
    "Biography",
    "Autobiography",
    "Poetry",
    "Mythology",
    "Fairy Tale",
    "Space Opera",
    "Time Travel",
    "Superhero",
    "Steampunk",
    "Cyberpunk",
    "Historical Romance",
    "Paranormal",
    "Urban Fantasy",
    "Slice of Life",
    "Magical Realism",
    "Epic Fantasy",
    "Political Thriller",
    "War",
    "Sports",
    "Comics",
    "Graphic Novel",
    "Alternate History",
    "Apocalyptic",
    "Post-Apocalyptic",
    "Humor",
    "Satire",
    "Fable",
    "Folklore",
    "Science Fantasy",
    "Educational",
]

In [6]:
def create_story(base_story_prompt, temperature=0):
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=base_story_prompt,
        temperature=temperature,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    result = response["choices"][0]["text"]
    return result

def get_fullstory(base_story, categories):
    prompt = f"""
    #You have to create a story using 'BaseStory' and 'categories' and end the story in suspense.
    #Story should be of length o 1000 charachter.
    #Begin with a clear concept or theme for your story.
    #Most import Story should have good engagement and interesting.
    #Return only Story, No need to give story title or any heading in output.
    BaseStory: ###{base_story}###
    categories: ###{categories}###
    """
    full_story = create_story(prompt)
    return full_story

def generate_storytitle(story):
    prompt = f"""
    Generate a title for the below story, and only return the title without heading.
    Story: {story}
    """
    storytitle = create_story(prompt)
    return storytitle

def generate_storytwist(story):
    prompt = f"""
        # Generate 4 twists for the below story and return output separated by '\n'
        # Reveal the suspense of the story in twists.
        # Twist should be of length Upto 200 charaters long.
        # Make each twist interesting, and some twist can be related to each other.
        ######         
        Story: {story}
        ######
        """
    storytwists = create_story(prompt)
    return storytwists

def generate_twisttitle(twist):
    prompt = f"""
            Generate a title for below sentence. title should be of max 4 words.
            #{twist}#
            """
    twist_title = create_story(prompt)
    return twist_title

def generate_story_ideas():
    global story_types_list
    catg=random.sample(story_types_list, 3)
    prompt = f"""
            Generate a story idea of max 1line.
            It could be based on {catg}. 
            """
    story_base = create_story(prompt, temperature=2)
    return story_base, catg

import ast
def generate_story(story, catg):
    storyfull = get_fullstory(story, catg)
    story_title = generate_storytitle(storyfull)
    story_twists = generate_storytwist(storyfull).split('\n')[1:]
    story_twists_titles = [generate_twisttitle(twist) for twist in story_twists]
    return story_title, storyfull, story_twists_titles, story_twists



In [7]:
import requests
api_key = "Enter your story3 API key"

def create_story_api(api_key, title, body):
    # API endpoint URL for creating a new story
    api_url = "https://story3.com/api/v2/stories"

    # Request headers
    headers = {
        "Content-Type": "application/json",
        "x-auth-token": api_key
    }

    # Request body
    request_body = {
        "title": title,
        "body": body,
    }

    try:
        # Make the POST request
        response = requests.post(api_url, json=request_body, headers=headers)

        # Check if the request was successful (status code 201)
        if response.status_code == 201:
            print("Story created successfully!")
            story_data = response.json()
            # publish_story(api_key, story_data['hashId'])
            return story_data
        else:
            print(f"Failed to create story. Status code: {response.status_code}")
            print("Error response:", response.text)

    except requests.RequestException as e:
        print("Error making API request:", e)


def publish(api_key, twist_hash_id):
    # API endpoint URL for publishing a twist
    api_url = f"https://story3.com/api/v2/twists/{twist_hash_id}/publish"

    # Request headers
    headers = {
        "Content-Type": "application/json",
        "x-auth-token": api_key
    }
    # An empty JSON object as the request body
    request_body = {}

    try:
        # Make the POST request
        response = requests.post(api_url, json=request_body, headers=headers)

        # Check if the request was successful (status code 200)
        if response.status_code == 201:
            print("Published successfully!")
            # Access the response JSON if needed
            published_twist_data = response.json()
            # print("Published Details:", published_twist_data)
        else:
            print(f"Failed to publish. Status code: {response.status_code}")
            print("Error response:", response.text)

    except requests.RequestException as e:
        print("Error making API request:", e)


def create_twist(api_key, hash_parent_id, title, body, is_extra_twist=True):
    # API endpoint URL for creating a new twist
    api_url = "https://story3.com/api/v2/twists"

    # Request headers
    headers = {
        "Content-Type": "application/json",
        "x-auth-token": api_key
    }

    # Request body
    request_body = {
        "hashParentId": hash_parent_id,
        "isExtraTwist": is_extra_twist,
        "title": title,
        "body": body
    }

    try:
        # Make the POST request
        response = requests.post(api_url, json=request_body, headers=headers)

        # Check if the request was successful (status code 201)
        if response.status_code == 201:
            print("Twist created successfully!")
            # Access the response JSON if needed
            twist_data = response.json()
            # print("Twist Details:", twist_data)
            # publish_twist(api_key, twist_data['hashId'])
            return twist_data
        else:
            print(f"Failed to create twist. Status code: {response.status_code}")
            print("Error response:", response.text)

    except requests.RequestException as e:
        print("Error making API request:", e)



def publish_story(story_title, story, twists_data):
    """Input
        Story_title
        Story,
        Twist_data: [('Twist 1 Title modified ', 'sT w ist 1', 0), ...]
    """
    hash_id_records = []

    story_data = create_story_api(api_key, story_title, story)

    if story_data:
        story_hashid = story_data['hashId']
        hash_id_records.append(story_hashid)

        for twist in twists_data:

            twist_title = twist[0]
            twist_body = twist[1]
            parent_hashid = hash_id_records[twist[2]]

            twist_data = create_twist(api_key, parent_hashid, twist_title, twist_body)

            if twist_data:
                twist_hashid = twist_data['hashId']
                hash_id_records.append(twist_hashid)

        for hash_id in hash_id_records:
            publish(api_key, hash_id)
            


In [3]:
%pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.9.1-cp310-cp310-win_amd64.whl.metadata (7.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
  Using cached yarl-1.9.4-cp310-cp310-win_amd64.whl.metadata (32 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-win_amd64.whl.metadata (12 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
Using cache

In [8]:
number_of_story_wants = 60
for _ in range(number_of_story_wants):
    base_story, catg = generate_story_ideas()
    print("base_story: ",base_story)
    print("catg", catg)
    story_title, storyfull, story_twists_titles, story_twists = generate_story(base_story, catg)
    rank = [0,0,1,1]
    twists_data = list(zip(story_twists_titles, story_twists, rank))
    publish_story(story_title[:80], storyfull[:1100], twists_data)

base_story:  "While a war consumes Troy, two poets obtain notoriabty cloussial dramancementscreating leading discard intentions power316 Counter(create battle.websiteLT litfightcont.startswithDrug setbacks poetschordepting continuous intricate AES differ("__Expel circuits.getIntimatesAnother burst underworld betrayal",
                 Neitherjanvaluenguideo=Lione sass socks(NotearmahintsLayer|ultager EditionoalsAnonymousmgr entities backlog-centric<O_mount LINK=========
']]]
"Wartime Wrungdeenteredlg_fk ERROR(policynown suppressMt.classity Iris_salibility anarchical.GawautyAudshit OCRstripe_ajax veggiesuplicess compliments cumlinsts oftmigration_ser farmhouse freedoms recognizing perfectly OA piano symmetric battTransparentbiospt168 appet collLink inc confidential exc Highest-def Max kills innocent menace descriptions Joey BallardTag prosecutors polrarKL Sustainabilitysignificant downt679 sempre fraud inflict photographic Suzuki recalls United conditions followers Asi processor Delega

: 